In [5]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [4]:

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
print(device)

cpu


In [7]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), 
    transforms.Normalize([0.5,0.5,0.5], 
                        [0.5,0.5,0.5])
])

In [21]:
train_path=r'C:/Users/sneha/Desktop/p/train'
test_path=r'C:/Users/sneha/Desktop/p'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [22]:

root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [23]:
print(classes)

['jute', 'maize', 'rice', 'sugarcane', 'wheat']


In [24]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        self.bn1=nn.BatchNorm2d(num_features=12)
        self.relu1=nn.ReLU()
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        
        self.relu2=nn.ReLU()
        
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        
        self.bn3=nn.BatchNorm2d(num_features=32)
    
        self.relu3=nn.ReLU()
        
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            

In [25]:
model=ConvNet(num_classes=6).to(device)

In [26]:

optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [27]:
num_epochs=10

In [39]:

train_count=len(glob.glob(train_path+'/**/*.jpeg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [40]:
print(train_count,test_count)

201 44


In [41]:


best_accuracy=0.0

for epoch in range(num_epochs):
    
    
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(44.6731) Train Accuracy: 0.3482587064676617 Test Accuracy: 0.18181818181818182
Epoch: 1 Train Loss: tensor(19.0381) Train Accuracy: 0.4427860696517413 Test Accuracy: 2.090909090909091
Epoch: 2 Train Loss: tensor(16.1160) Train Accuracy: 0.30845771144278605 Test Accuracy: 0.5454545454545454
Epoch: 3 Train Loss: tensor(7.9357) Train Accuracy: 0.5920398009950248 Test Accuracy: 0.2727272727272727
Epoch: 4 Train Loss: tensor(6.0896) Train Accuracy: 0.572139303482587 Test Accuracy: 1.1818181818181819
Epoch: 5 Train Loss: tensor(4.2153) Train Accuracy: 0.6766169154228856 Test Accuracy: 0.5
Epoch: 6 Train Loss: tensor(4.4506) Train Accuracy: 0.6865671641791045 Test Accuracy: 1.4318181818181819
Epoch: 7 Train Loss: tensor(3.6529) Train Accuracy: 0.7064676616915423 Test Accuracy: 0.25
Epoch: 8 Train Loss: tensor(2.8781) Train Accuracy: 0.7412935323383084 Test Accuracy: 0.22727272727272727
Epoch: 9 Train Loss: tensor(1.2416) Train Accuracy: 0.8407960199004975 Test Accu